In [75]:
import json
import os


class DST():
    def __init__(self):
        self.state = json.load(open('dictionary.json'))

    def update(self, user_act=None):
        for intent, domain, slot, value in user_act:
            domain = domain.lower()
            intent = intent.lower()
            slot = slot.lower()
            
            if domain not in self.state['belief_state']:
                continue

            if intent == 'inform':
                if slot == 'none' or slot == '':
                    continue

                domain_dic = self.state['belief_state'][domain]

                if slot in domain_dic:
                    self.state['belief_state'][domain][slot] = value

            elif intent == 'request':
                if domain not in self.state['request_state']:
                    self.state['request_state'][domain] = {}
                if slot not in self.state['request_state'][domain]:
                    self.state['request_state'][domain][slot] = 0
                    
        self.state['user_act'] = user_act
        return self.state

In [109]:
dst = DST()
user_act = [('inform', 'payment', 'type', 'karta'), ('inform', 'delivery', 'type','paczkomat'), ('inform', 'product', 'type', 'telefon'), ('request', 'product', 'name', '?')]
state = dst.update(user_act)

In [110]:
print(state['belief_state'])
print(state['request_state'])
dst.state

{'payment': {'type': 'karta', 'amount': '', 'loyalty_card': ''}, 'delivery': {'type': 'paczkomat', 'address': '', 'time': ''}, 'product': {'name': '', 'type': 'telefon', 'brand': '', 'price': '', 'quantity': '', 'quality': ''}}
{'product': {'name': 0}}


{'user_act': [('inform', 'payment', 'type', 'karta'),
  ('inform', 'delivery', 'type', 'paczkomat'),
  ('inform', 'product', 'type', 'telefon'),
  ('request', 'product', 'name', '?')],
 'system_act': [],
 'belief_state': {'payment': {'type': 'karta',
   'amount': '',
   'loyalty_card': ''},
  'delivery': {'type': 'paczkomat', 'address': '', 'time': ''},
  'product': {'name': '',
   'type': 'telefon',
   'brand': '',
   'price': '',
   'quantity': '',
   'quality': ''}},
 'request_state': {'product': {'name': 0}},
 'terminated': False,
 'history': []}

In [122]:
from collections import defaultdict
import json




class SimpleRulePolicy():
    def __init__(self):
        with open('product_db.json', encoding='utf-8') as json_file:
            self.db =  json.load(json_file)

    def predict(self, state):
        self.results = []
        system_action = defaultdict(list)
        user_action = defaultdict(list)
        system_acts = []
        for intent, domain, slot, value in state['user_act']:
            user_action[(domain.lower(), intent.lower())].append((slot.lower(), value))
        print(user_action)
        for user_act in user_action:
            self.update_system_action(user_act, user_action, state, system_action)
            print(self.update_system_action(user_act, user_action, state, system_action))
        system_acts = [[intent, domain, slot, value] for (domain, intent), slots in system_action.items() for slot, value in slots]
        state['system_act'] = system_acts
        return system_acts


    def update_system_action(self, user_act, user_action, state, system_action):
        domain, intent = user_act

        # Reguła 1
        if intent == 'request':
            if len(self.results) == 0:
                system_action[(domain, 'NoOffer')] = []
            else:
                for slot in user_action[user_act]:                  
                    if slot[0] in self.results[0]:
                        system_action[(domain, 'Inform')].append([slot[0], self.results[0].get(slot[0], 'unknown')])

        # Reguła 2
        elif intent == 'inform':
            if len(self.results) == 0:
                system_action[(domain, 'NoOffer')] = []
            else:
                system_action[(domain, 'Inform')].append(['Choice', str(len(self.results))])
                choice = self.results[0]

                if domain in ["payment", "delivery", "product"]:
                    system_action[(domain, 'Recommend')].append(['Name', choice['name']])

In [123]:
policy = SimpleRulePolicy()
policy.predict(dst.state)

defaultdict(<class 'list'>, {('payment', 'inform'): [('type', 'karta')], ('delivery', 'inform'): [('type', 'paczkomat')], ('product', 'inform'): [('type', 'telefon')], ('product', 'request'): [('name', '?')]})
None
None
None
None


[]